### Космический кризис
На далёкой планете Видеосфера-IX правительство ввело новый закон: безопасность на улицах теперь обеспечивают интеллектуальные патрули-роботы, способные видеть всё происходящее через камеры городского видеонаблюдения. Каждый робот оснащён отличным детектором объектов — он мгновенно находит граждан, гравибайки и потерянные зонтики на каждом кадре, но… абсолютно не понимает, что одно и то же существо на 10 разных кадрах — всё ещё тот же гражданин, а не 10 разных.

Государственные инженеры спешно посчитали, что из-за этого повторного подсчёта у них уже начинается кризис — они думают, что на улицах скопилось гораздо больше людей, чем есть на самом деле, и требуют срочно наладить учёт перемещающихся объектов во времени.

Однако финансирования на доработку системы не выделили — всё ушло на запуск межгалактической лиги по квантовым шахматам среди студентов университета Планеты Видеосфера.

Задача для вас: имея выходы детектора людей (bounding boxes), разработать трекинг-модуль: такой алгоритм, который сможет сопоставлять одного и того же объекта на разных кадрах — «сшивать» дорожку его перемещения по видео.

### Комментарии к задаче
- не использовать готовые e2e трекинг модели
- необходимо предсказать только idx-ы людей с разных кадров. Один человек - один индекс на всей видеодорожке (номер индекса не имеет значения)
- для оценки решения используется метрика МОТА
    - $$MOTA =1 - \frac{\sum_t{(FN_t+FP_t+IDSW_t)}}{\sum{GT_t}}$$
    - IDSW_t (index switch) - число некоретно предсказанных индексов людей
- данные - https://drive.google.com/file/d/116ObQT_Ma9Wi8aOIUT5QvrQHtmAKXdnt/view?usp=sharing

In [1]:
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Only GPU 1 will be visible to PyTorch

import torch
import cv2
import os
from glob import glob
from IPython.display import Video
import numpy as np

DATA_PATH = "/home/kashindanil/temp_project"
DIR_TO_VIZ = os.path.join(DATA_PATH, "mp4data/")
os.makedirs(DIR_TO_VIZ, exist_ok=True)


### Vizualization

In [11]:
train_df = pd.read_csv(os.path.join(DATA_PATH, 'data/MOT17/TEST/labels.csv'))
train_df['vid_path'] = train_df['vid_path'].apply(lambda x: os.path.join(DATA_PATH, x))
train_df['full_path'] = train_df.apply(lambda row: os.path.join(row['vid_path'], row['image_name']), axis=1)

In [12]:
def get_color(id):
    """Generate a color from object_id (as a tuple in BGR)."""
    np.random.seed(id)
    return tuple(int(x) for x in np.random.randint(0, 255, 3))

def images_to_mp4(image_pathes = None, list_bboxes = None, idxes = None, output_video_path = "output.mp4", fps=24, pattern="*.jpg"):
    frame = cv2.imread(image_pathes[0])
    height, width, layers = frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    for idx, image_file in enumerate(image_pathes):
        frame = cv2.imread(image_file)
        if list_bboxes:
            bboxes = list_bboxes[idx]
            for box_idx, box in enumerate(bboxes):
                color = get_color(idxes[idx][box_idx]) if idxes else (0, 255, 0)
                cv2.rectangle(frame, (box[0], box[1]), (box[0]+box[2], box[1]+box[3]), color, 2)
                if idxes:
                    cv2.putText(frame, str(idxes[idx][box_idx]), (box[0], box[1]-5),  cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        video.write(frame)
    
    video.release()
    print(f"Video saved as {output_video_path}")


def decode_video(path_to_orig_video, save_path):
    os.system(f"ffmpeg -y -i {path_to_orig_video} {save_path}")

In [13]:
vid_path = train_df['vid_path'].unique()[0]

In [14]:
images_to_mp4(image_pathes = train_df[train_df['vid_path'] == vid_path]['full_path'].drop_duplicates().tolist(),
               output_video_path = os.path.join(DIR_TO_VIZ, 'test_video1.mp4'))
decode_video(os.path.join(DIR_TO_VIZ, 'test_video1.mp4'), os.path.join(DIR_TO_VIZ, 'decoded_test_video1.mp4'))

Video saved as /home/kashindanil/temp_project/mp4data/test_video1.mp4


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

In [16]:
Video(os.path.join(DIR_TO_VIZ, 'decoded_test_video1.mp4')) 

### Расчет метрик

In [3]:
gt_df = pd.read_csv(os.path.join(DATA_PATH, 'data/MOT17/TEST/labels.csv'))
gt_df['vid_path'] = gt_df['vid_path'].apply(lambda x: os.path.join(DATA_PATH, x))
result_df = pd.read_csv('data/MOT17/sample_submission.csv')
result_df

,frame_number,object_id,bb_left,bb_top,bb_width,bb_height,cls,conf,image_name,vid_path
0,1.0,1.0,627.215820,577.873901,89.832092,262.703369,0.0,0.854258,000001.jpg,data/MOT17/TEST/MOT17-02-SDP/img1/
1,1.0,2.0,1417.526367,597.761108,163.823730,366.248352,0.0,0.833204,000001.jpg,data/MOT17/TEST/MOT17-02-SDP/img1/
2,1.0,3.0,1521.740845,603.622253,154.183350,334.185181,0.0,0.780781,000001.jpg,data/MOT17/TEST/MOT17-02-SDP/img1/
3,1.0,4.0,503.618896,579.919189,96.123779,282.286865,0.0,0.645980,000001.jpg,data/MOT17/TEST/MOT17-02-SDP/img1/
4,1.0,5.0,747.827026,511.795532,31.721252,109.744263,0.0,0.480551,000001.jpg,data/MOT17/TEST/MOT17-02-SDP/img1/
...,...,...,...,...,...,...,...,...,...,...
10783,1125.0,5.0,1315.846802,553.570679,57.596191,212.237091,0.0,0.630653,001125.jpg,data/MOT17/TEST/MOT17-02-SDP/img1/
10784,1125.0,6.0,1372.951050,549.365479,75.153809,221.289062,0.0,0.599729,001125.jpg,data/MOT17/TEST/MOT17-02-SDP/img1/
10785,1125.0,7.0,160.488617,617.710022,85.917282,258.122681,0.0,0.589529,001125.jpg,data/MOT17/TEST/MOT17-02-SDP/img1/
10786,1125.0,8.0,1700.870361,539.333252,65.721191,196.503387,0.0,0.552905,001125.jpg,data/MOT17/TEST/MOT17-02-SDP/img1/


In [9]:
import motmetrics as mm

def compute_one_frame(acc, GT_labels, GT_bboxes, predict_labels, predict_bboxes, max_iou=0.5):
    dists = mm.distances.iou_matrix(GT_bboxes, predict_bboxes)
    acc.update(GT_labels, predict_labels, dists)

def compute_dataset(gt_dataset, pred_dataset):
    acc_list = []
    for vid_path in sorted(gt_dataset['vid_path'].unique()):
        model_name_acc = mm.MOTAccumulator(auto_id=True)
        gt_for_vid = gt_dataset[gt_dataset['vid_path']==vid_path]
        image_names = gt_for_vid['image_name'].unique()
        for im_name in image_names:
            gt_idxes = gt_for_vid[gt_for_vid['image_name']==im_name]['object_id'].tolist()
            gt_bboxes = gt_for_vid[gt_for_vid['image_name']==im_name][['bb_left', 'bb_top', 'bb_width', 'bb_height']].to_numpy()    
            pred_idxes = pred_dataset[pred_dataset['image_name']==im_name]['object_id'].tolist()
            pred_bboxes = pred_dataset[pred_dataset['image_name']==im_name][['bb_left', 'bb_top', 'bb_width', 'bb_height']].to_numpy()    
            compute_one_frame(model_name_acc, gt_idxes, gt_bboxes, pred_idxes, pred_bboxes)
        acc_list.append(model_name_acc)
    
    mh = mm.metrics.create()
    summary = mh.compute_many(
        acc_list,
        metrics=['num_frames', 'num_matches', 'num_switches',
                    'num_false_positives', 'num_misses', 'mota',
                    'idf1', 'precision', 'recall'],
        names=sorted(gt_dataset['vid_path'].unique()))
    return summary


In [10]:
summary = compute_dataset(gt_df, result_df)

In [11]:
summary.T

,/home/kashindanil/temp_project/data/MOT17/TEST/MOT17-02-SDP/img1/,/home/kashindanil/temp_project/data/MOT17/TEST/MOT17-09-SDP/img1/
num_frames,600.000000,525.000000
num_matches,5709.000000,4020.000000
num_switches,29.000000,442.000000
num_false_positives,0.000000,451.000000
num_misses,12843.000000,863.000000
mota,0.307249,0.670235
idf1,0.462766,0.627662
precision,1.000000,0.908203
recall,0.308810,0.837934


In [12]:
summary['mota'].mean()

0.4887420412542167

### Vizualize preds

In [14]:
one_vid = gt_df[gt_df['vid_path']==gt_df['vid_path'][0]]

In [15]:

formated_one_vid = pd.merge(one_vid.groupby('frame_number')[['object_id', 'bb_left', 'bb_top', 'bb_width', 'bb_height', ]].apply(
                                        lambda x: x.values
                                    ).reset_index(name='idx_and_bboxes'), 
                                    one_vid[['image_name', 'vid_path', 'frame_number']].drop_duplicates(), on = 'frame_number', how='left')
formated_one_vid['object_idxes'] = formated_one_vid['idx_and_bboxes'].apply(lambda x: [i[0] for i in x])
formated_one_vid['bboxes'] = formated_one_vid['idx_and_bboxes'].apply(lambda x: [i[1:] for i in x])
formated_one_vid['full_path'] = formated_one_vid.apply(lambda row: os.path.join(row['vid_path'], row['image_name']), axis=1)


In [16]:
images_to_mp4(formated_one_vid['full_path'].tolist(), 
              formated_one_vid['bboxes'].tolist(), 
              formated_one_vid['object_idxes'].tolist(),
              output_video_path = os.path.join(DIR_TO_VIZ, 'test_video1.mp4'))
decode_video(os.path.join(DIR_TO_VIZ, 'test_video1.mp4'), os.path.join(DIR_TO_VIZ, 'decoded_test_video1.mp4'))

Video saved as /home/kashindanil/temp_project/mp4data/test_video1.mp4


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

In [17]:
Video(os.path.join(DIR_TO_VIZ, 'decoded_test_video1.mp4')) 